# Requirements
For this tutorial, you will need `python >= 3.9`.

Make sure to install `spatialdata_io >= 0.0.9` and `spatialdata_xenium_explorer >= 1.0.4`, i.e.:
```sh
pip install spatialdata-io
pip install spatialdata_xenium_explorer
```

In [1]:
import spatialdata_io
import spatialdata_xenium_explorer

# 1. Download tutorial data

This tutorial is based on a Xenium sample of human skin, which can be downloaded on this [10x Genomics webpage](https://www.10xgenomics.com/datasets/human-skin-preview-data-xenium-human-skin-gene-expression-panel-add-on-1-standard).
In particular, download the following directory/file:
- [Xenium Output Bundle (full)](https://cf.10xgenomics.com/samples/xenium/1.7.0/Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE/Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_outs.zip) (unzip it into a directory)
- [Supplemental: Post-Xenium H&E image (OME-TIFF)](https://cf.10xgenomics.com/samples/xenium/1.7.0/Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE/Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_he_image.ome.tif) (move it into the latter directory)

# 2. Read the Xenium data

In [2]:
# directory containing the Xenium data you downloaded
data_path = "Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_outs"

The Xenium data can be read by `spatialdata-io`:

In [3]:
sdata = spatialdata_io.xenium(data_path)

INFO     reading Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_outs/cell_feature_matrix.h5                            


/Users/quentinblampey/Library/Caches/pypoetry/virtualenvs/spatialdata-xenium-explorer-yAxRBYoR-py3.9/lib/python3.9/site-packages/spatialdata/models/models.py:725: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(adata.obs[region_key]):
/Users/quentinblampey/Library/Caches/pypoetry/virtualenvs/spatialdata-xenium-explorer-yAxRBYoR-py3.9/lib/python3.9/site-packages/spatialdata_io/readers/xenium.py:197: UserWarning: Converting `region_key: region` to categorical dtype.
  table = TableModel.parse(adata, region=specs["region"], region_key="region", instance_key=str(XeniumKeys.CELL_ID))
/Users/quentinblampey/Library/Caches/pypoetry/virtualenvs/spatialdata-xenium-explorer-yAxRBYoR-py3.9/lib/python3.9/site-packages/spatialdata/models/models.py:620: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [4]:
sdata

SpatialData object with:
├── Images
│     ├── 'morphology_focus': MultiscaleSpatialImage[cyx] (1, 13725, 28467), (1, 6862, 14233), (1, 3431, 7116), (1, 1715, 3558), (1, 857, 1779)
│     └── 'morphology_mip': MultiscaleSpatialImage[cyx] (1, 13725, 28467), (1, 6862, 14233), (1, 3431, 7116), (1, 1715, 3558), (1, 857, 1779)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 10) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (87499, 1) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (87499, 1) (2D shapes)
└── Table
      └── AnnData object with n_obs × n_vars = 87499 × 382
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'
    obsm: 'spatial': AnnData (87499, 382)
with coordinate systems:
▸ 'global', with

# 3. Image alignment

### Alignment from the Xenium Explorer to `SpatialData`

Update the `SpatialData` object

In [5]:
from spatialdata_xenium_explorer.core.images import ome_tif
from pathlib import Path

In [6]:
h_and_e = ome_tif(Path(data_path) / "Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_he_image.ome.tif")

[INFO] (spatialdata_xenium_explorer.core.images) Transformed 4D image into a 3D image of shape (c, y, x) = (3, 43287, 22209)
[WARNING] (spatialdata_xenium_explorer.core.images) Channel names couldn't be read. Using ['0', '1', '2'] instead.


In [8]:
image_key = "morphology_mip"

spatialdata_xenium_explorer.align(sdata, h_and_e, "Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_he_imagealignment.csv", image_key=image_key)

INFO     `dims` is specified redundantly: found also inside `data`.                                                


[INFO] (spatialdata_xenium_explorer.core.images) Adding image image:
<xarray.SpatialImage 'image' (c: 3, y: 43287, x: 22209)>
dask.array<rechunk-merge, shape=(3, 43287, 22209), dtype=uint8, chunksize=(1, 4096, 4096), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U1 '0' '1' '2'
  * y        (y) float64 0.5 1.5 2.5 3.5 ... 4.328e+04 4.329e+04 4.329e+04
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 2.221e+04 2.221e+04 2.221e+04
Attributes:
    transform:  {'global': Affine (x, y -> x, y)\n    [1.83698152e-03 6.44010...


In [9]:
sdata

SpatialData object with:
├── Images
│     ├── 'Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_he_image': SpatialImage[cyx] (3, 43287, 22209)
│     ├── 'morphology_focus': MultiscaleSpatialImage[cyx] (1, 13725, 28467), (1, 6862, 14233), (1, 3431, 7116), (1, 1715, 3558), (1, 857, 1779)
│     └── 'morphology_mip': MultiscaleSpatialImage[cyx] (1, 13725, 28467), (1, 6862, 14233), (1, 3431, 7116), (1, 1715, 3558), (1, 857, 1779)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 10) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (87499, 1) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (87499, 1) (2D shapes)
└── Table
      └── AnnData object with n_obs × n_vars = 87499 × 382
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 's

### Alignment from `SpatialData` to the Xenium Explorer

In [24]:
from spatialdata.transformations import get_transformation, Affine
import numpy as np

In [29]:
spatial_element = sdata["Xeniumranger_V1_hSkin_Melanoma_Add_on_FFPE_he_image"]

transformation = get_transformation(spatial_element, to_coordinate_system="global")

In [30]:
assert isinstance(transformation, Affine), "The Xenium Explorer only supports Affine transformations"

np.savetxt("alignment_matrix.csv", transformation.matrix, delimiter=",")

# 4. Update the cells on the Explorer

### Update the cell categories

In [ ]:
import scanpy as sc

sc.pp.normalize_total(sdata.table)
sc.pp.log1p(sdata.table)
sc.pp.pca(sdata.table)
sc.pp.neighbors(sdata.table)
sc.tl.leiden(sdata.table)

Now, you can update the explorer with your new cluster assignment. You don't need to re-run the complete conversion; you can edit the `analysis.zarr.zip` file only, as below.

> Alternatively, you can use the `spatialdata_xenium_explorer` CLI instead of the API, as detailed [here](https://quentinblampey.github.io/spatialdata_xenium_explorer/cli/#update-obs).

In [ ]:
spatialdata_xenium_explorer.write_cell_categories(explorer_path, sdata.table)

To visualize these clusters, re-open the `experiment.xenium` file and select the new `"leiden"` cell group (under the "Cells" panel and in the "Cell groups" dropdown). See the examples above to see how it looks on the Xenium Explorer.

### Update the cells boundaries

# 5. Select cells from the lasso tool